# o3/o4-mini 関数呼び出しガイド

## はじめに
o3/o4-miniモデルは、応答する前により長時間思考するように訓練された、私たちのoシリーズモデルの最新版です。これらは私たちがこれまでにリリースした中で最も賢いモデルであり、ツール呼び出し機能においてo1/o3-miniから大幅な進歩を表しています。これらのモデルは、思考の連鎖（CoT）内でネイティブにツールを使用するように訓練されており、いつ、どのようにツールを使用するかについての推論能力の向上を実現しています。私たちは、responses APIを介してこれらのモデルで[関数を呼び出す](https://cookbook.openai.com/examples/reasoning_function_calls)方法に関するガイドをリリースしており、このガイドはそれを基に構築され、これらのモデルで最高の関数呼び出しパフォーマンスを得る方法をお伝えします。

## より良い関数呼び出しパフォーマンスのためのプロンプトガイダンス
o3/o4-miniモデルの背後にある関数呼び出しインテリジェンスを最大限に活用するために、開発者プロンプトと関数の説明の両方において、いくつかのベストプラクティスを推奨します。

### 推論モデルにおける開発者プロンプト、システムプロンプト、関数説明に関する簡単な注意事項
開発者からの指示であることを推論モデルに明示するために、開発者メッセージを導入しました。oシリーズモデルでは、開発者が提供したシステムメッセージは、内部的に自動的に開発者メッセージに変換されます。実用的な目的では、開発者プロンプトを従来のシステムプロンプトと同様のものとして扱うことができますが、明確性と正確性のため、このガイドではそのような指示をすべて開発者プロンプト/メッセージと呼びます。

このドキュメントで関数説明と言う場合、APIリクエストのtoolパラメータ内の各関数オブジェクトのdescriptionフィールドにある説明テキストを意味します。この説明は、いつ、どのように関数を使用するかをモデルに伝えます。以下は、関数呼び出し[ドキュメント](https://platform.openai.com/docs/guides/function-calling)からの例です：

```
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]
```
ここで、`"Get current temperature for provided coordinates in celsius."`が関数説明として機能します。

定義を明確にしたところで、ベストプラクティスについて説明を始めることができます。

### 開発者メッセージによるコンテキスト設定
1. **一般的なコンテキスト**: 一般的なプロンプトエンジニアリングのベストプラクティスに従い、ロールプロンプティングは基本的な動作、トーン、および可能なアクションセットを設定するのに役立ちます。例えば：
```
You are an AI retail agent.

As a retail agent, you can help users cancel or modify pending orders, return or exchange delivered orders, modify their default user address, or provide information about their own profile, orders, and related products.
```
2. **関数呼び出しの順序**: o3/o4-miniはツールを使って目標を達成するように訓練されています。しかし、ツール呼び出しの順序で間違いを犯すことがあります。このようなケースを防ぐため、特定のタスクを達成するための順序を明示的に示すことが推奨されます。例えば、コーディングエージェントがまだ存在しないディレクトリにファイルを作成する可能性がある失敗ケースを防ぐため、以下を追加すれば通常十分です：
```
check to see if directories exist before making files
```
大量で明確に定義されたタスクの場合、呼び出す関数の順序を明示的に示すことで、さらに堅牢にできます。例えば：
```
To Process a refund for a delivered order, follow the following steps:
1. Confirm the order was delivered. Use: `order_status_check`
2. Check the refund eligibility policy. Use: `refund_policy_check`
3. Create the refund request. Use: `refund_create`
4. Notify the user of refund status. Use: `user_notify`
```

3. **ツール使用の境界の定義**: モデルが特定のツールをいつ呼び出すべきか、いつ呼び出すべきでないかの境界を明確にすることが役立ちます。これは開発者プロンプトレベルとツール説明レベルの両方で行うことができます。以下は開発者プロンプトの例です：
```
Be proactive in using tools to accomplish the user's goal. If a task cannot be completed with a single step, keep going and use multiple tools as needed until the task is completed. Do not stop at the first failure. Try alternative steps or tool combinations until you succeed.

- Use tools when:
  - The user wants to cancel or modify an order.
  - The user wants to return or exchange a delivered product.
  - The user wants to update their address or contact details.
  - The user asks for current or personalized order or profile info.

- Do not use tools when:
  - The user asks a general question like "What's your return policy?"
  - The user asks something outside your retail role (e.g., "Write a poem").

If a task is not possible due to real constraints (For example, trying to cancel an already delivered order), explain why clearly and do not call tools blindly.
```

### 関数の説明
関数の説明では、いつ呼び出すべきか、引数をどのように構築すべきかを明確にする必要があります。

関数の説明は、関数をいつ呼び出すべきか、引数をどのように構築すべきかを明確にする理想的な場所です。これは推論モデルとツールAPI間の持続可能なインターフェース契約として機能します。

一般的に、関数の説明は何をするか、どのように呼び出すかを定義します。開発者の指示は、ツールを使用するエージェントにガイダンスを提供します。そのため、類似の目的で使用できる複数のツールがある場合、開発者は指示でそれらを区別できます。エージェントワークフローの要件で特定の順序でツールを使用する優先順位がある場合、または特定のツールを頻繁に使用する対控えめに使用する場合も、これらは開発者の指示に含まれます。

適切に構造化された説明は、重要な基準と引数要件を早期に固定することで精度を向上させ、誤動作を減らすことができます。また、開発者がツール定義自体により近い場所で、開発者プロンプトの外で「積極性」制御ヒューリスティックをエンコードすることも可能にします。

1. **使用基準**: 開発者プロンプトを通じて関数呼び出しの積極性を調整できるのと同様に、関数説明レベルで関数がどのように呼び出されるかをさらに調整できます。以下は`file_create`関数の例です：
```
Creates a new file with the specified name and contents in a target directory. This function should be used when persistent storage is needed and the file does not already exist.
- Only call this function if the target directory exists. Check first using the `directory_check` tool.  
- Do not use for temporary or one-off content—prefer direct responses for those cases.  
- Do not overwrite existing files. Always ensure the file name is unique.
- Do not overwrite existing files.  
  If replacement is intended and confirmed, use `file_delete` followed by `file_create`, or use `file_update` instead.
```
2. **Few-shotプロンプティング**: 推論モデルは非推論モデルほどfew-shotプロンプティングの恩恵を受けませんが、特にモデルが関数引数を正確に構築するのに苦労する場合、few-shotプロンプティングがツール呼び出しのパフォーマンスを向上させることがわかりました。例えば、以下はツール説明として渡されるgrepツールのツール説明の例です：

```
Use this tool to run fast, exact regex searches over text files using the `ripgrep` engine.


- Always escape special regex characters: ( ) [ ] { } + * ? ^ $ | . \\
- Use `\\` to escape any of these characters when they appear in your search string.
- Do NOT perform fuzzy or semantic matches.
- Return only a valid regex pattern string.

Examples:
Literal            -> Regex Pattern         
function(          -> function\\(           
value[index]       -> value\\[index\\]      
file.txt           -> file\\.txt            
user|admin         -> user\\|admin          
path\to\file       -> path\\\\to\\\\file     
```

3. **重要なルールを前面に出し、注意散漫を最小限に抑える**: 上記の例では、特殊文字をエスケープする指示が、モデルが読む最初のものの一つであることに注目してください。**より悪い**代替案は以下のようなものです：
```
Performs a fast regex-based text search that looks for exact pattern matches within files or entire directories, leveraging the ripgrep tool for high-speed scanning.
Output follows ripgrep formatting and can optionally display line numbers and matched lines.
To manage verbosity, results are limited to a maximum of 50 hits.
You can fine-tune the search by specifying inclusion or exclusion rules based on file types or path patterns.
This method is ideal when searching for literal text snippets or specific regular expressions.
It offers more accuracy than semantic methods when the goal is to locate a known string or structure.
It's generally recommended over semantic search when you're looking for a specific identifier—such as a function name, variable, or keyword—within a defined set of directories or file types.
```

これは、プロンプトの多くが規範的でなく、引数の構築方法に関する最も重要なルールが前面に出ていないため、パフォーマンスが悪くなります。前のプロンプトは、上記のものと比較して、このripgrepツールを使用するツール呼び出し精度評価で6%高いスコアを記録しました。

### 関数呼び出しの幻覚に対する防御策
o3モデルは他のモデルよりも幻覚を起こしやすい可能性があることを認識しています。これらの幻覚は、実際には実行せずにバックグラウンドでツールを呼び出すと約束したり、将来のターンでツールを呼び出すと約束したりする形で現れる場合があります。このような場合、これらの種類の幻覚を最小限に抑えるために、いくつかの領域で明示的に指示することが有効です：

1. 明示的な指示：不可能な場合に将来の関数呼び出しを約束するなど、一般的な幻覚を避けるようにモデルに明示的に指示します。

```
後で関数を呼び出すと約束してはいけません。関数呼び出しが必要な場合は、今すぐ実行してください。そうでなければ、通常通り応答してください。
```

2. 不正な引数を早期にキャッチ：
`strict`を`true`に設定すると、関数呼び出しが[関数スキーマ](https://platform.openai.com/docs/guides/function-calling?api-mode=responses#strict-mode)に確実に準拠するようになります。可能な限りこれを有効にすることをお勧めします。

引数に追加の複雑な形式要件がある場合（例：有効なpythonコードなど）、以下の指示を追加することで、モデルに期待される形式を思い出させることができます。

```
呼び出しを送信する前に、引数を形式に対して検証してください。不確実な場合は、推測する代わりに明確化を求めてください。
```

3. 怠惰な行動に関する追加の注意点：
o3の稀な怠惰な行動のインスタンスを認識しています。例えば、タスクを完了するのに十分な時間がないと述べたり、別途フォローアップすると約束したり、より詳細な情報を提供するよう明示的に求められても簡潔な回答をしたりすることがあります。以下の手順がこの行動を改善するのに役立つことがわかっています：

    a. 関連のないトピックについては新しい会話を開始する：  
       新しいまたは関連のないトピックに切り替える際は、同じコンテキストで続けるのではなく、新しい会話スレッドを開始してください。これにより、モデルが現在の主題に集中し、以前の無関係なコンテキストに影響されることを防ぎます。これは時として不完全または怠惰な応答につながる可能性があります。例えば、以前にコードデバッグについて議論していて、今度は以前の会話コンテキストを必要としないドキュメント作成のベストプラクティスについて質問したい場合は、明確性と集中を確保するために新しい会話を開始してください。

    b. リストが長くなりすぎた場合は、関連のない過去のツール呼び出し/出力を破棄し、ユーザーメッセージでコンテキストとして要約する：  
       会話履歴にもはや関連のない過去のツール呼び出しや出力の長いリストが含まれている場合は、それらをコンテキストから削除してください。代わりに、重要な情報の簡潔な要約をユーザーメッセージの一部として提供してください。これにより、コンテキストを管理しやすく保ち、モデルが最も関連性の高い情報のみにアクセスできるようになります。例えば、ツール出力の長いシーケンスがある場合、主要な結果を要約し、次のメッセージにはその要約のみを含めることができます。

    c. 私たちは常にモデルを改善しており、将来のバージョンでこの問題が解決されることを期待しています。


### 思考の連鎖プロンプトの回避
これらのモデルは推論モデルであり、内部的な思考の連鎖を生成するため、ツール呼び出し間で計画し推論するよう明示的にプロンプトする必要はありません。したがって、開発者は各関数呼び出しの前にモデルにより広範囲な計画を求めることで追加の推論を誘発しようとすべきではありません。推論モデルにより多くの推論を求めることは、実際にはパフォーマンスを損なう可能性があります。

推論要約に関する簡単な補足：モデルはツールを呼び出す前に推論トークンを出力します。ただし、推論要約は要約を生成するために最小限の実質的な推論トークン数を必要とするため、これらが常に要約を伴うとは限りません。

# Responses API

### より良いパフォーマンスのための推論アイテム
私たちはresponses APIを使用することの利点を詳しく説明した[クックブック](https://cookbook.openai.com/examples/responses_api/reasoning_items)をリリースしました。このガイドでも主要なポイントをいくつか再度述べる価値があります。o3/o4-miniは両方とも、単一のターン内でツール呼び出し間で内部推論が持続するように訓練されています。そのため、推論時にこれらの推論アイテムをツール呼び出し間で持続させることで、ツールがいつ、どのように呼び出されるかについてより良い判断を行う形で、より高い知能とパフォーマンスを実現できます。Responsesでは、これらの推論アイテムを（状態管理を私たちに任せたくない場合は、暗号化されたコンテンツを通じて私たち自身または自分で維持される）持続させることができますが、Chat Completionではできません。responses APIに切り替えて、関数呼び出し間でモデルが推論アイテムにアクセスできるようにすることは、関数呼び出しで可能な限り最高のパフォーマンスを引き出す最も簡単な方法です。以下は、便宜上再掲載したクックブックの例で、私たちが一切状態を保持しない方法で`encrypted_content`を使用して推論アイテムを渡し戻す方法を示しています：

In [2]:
from openai import OpenAI
import requests
import json
client = OpenAI()


def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

context = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="o3",
    input=context,
    tools=tools,
    store=False,
    include=["reasoning.encrypted_content"] # Encrypted chain of thought is passed back in the response
)


context += response.output # Add the response to the context (including the encrypted chain of thought)
tool_call = response.output[1]
args = json.loads(tool_call.arguments)



result = get_weather(args["latitude"], args["longitude"])

context.append({                               
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="o3",
    input=context,
    tools=tools,
    store=False,
    include=["reasoning.encrypted_content"]
)

print(response_2.output_text)

The current temperature in Paris is about 18.8 °C.


## ホストされたツールを使ったエージェント体験

Responses APIは、ホストされた/組み込みツールのセットをサポートしています。最近、Responses APIに[新しいツールと機能](https://openai.com/index/new-tools-and-features-in-the-responses-api/)も追加され、外部サービスに接続するエージェントアプリケーションの構築がより簡単になりました。Responses APIの組み込みツールにより、開発者は単一のAPI呼び出しでより高機能なエージェントを作成できます。

同じセッション内でホストされたツールとカスタムツールを組み合わせて使用できます。これにより強力な構成パターンが可能になりますが、ツールルーティングの明確性も重要になります。以下に具体的な推奨事項をいくつか示します：

1. 開発者プロンプトでツール使用の境界を明示的に定義する：複数のツールが類似の役割を果たせる場合（例：pythonツールとカスタム計算機の両方）、どのツールが好ましく、いつ使用するかをモデルに指示します。これにより曖昧さが減り、精度が向上し、ツールの過度な使用や使用不足を避けられます：

```
You are a helpful research assistant with access to the following tools:
- python tool: for any computation involving math, statistics, or code execution
- calculator: for basic arithmetic or unit conversions when speed is preferred

Always use the python tool for anything involving logic, scripts, or multistep math. Use the calculator tool only for simple 1-step math problems.
```

2. 内部知識が不十分な場合を明確にする：o3/o4-miniモデルは多くの場合、タスクを自分で解決できますが、ツールがより信頼性の高い回答を提供する場合があります。システムプロンプトを使用して、ツールがより適切な場合にモデルが「自分で解決しようとする」ことを避けるよう誘導します。

```
You have access to a `code_interpreter`. Always prefer using `code_interpreter` when a user asks a question involving:
- math problems
- data analysis
- generating or executing code
- formatting or transforming structured text

Avoid doing these directly in your own response. Always use the tool instead.
```

3. 開発者プロンプトは一元化された永続的な契約として機能するため、ホストされたツールとカスタム関数を組み合わせたい場合は、カバレッジの重複、信頼性の期待値、フォールバック動作を含めて、ツールの決定境界をここで明記します：

```
Use `python` for general math, data parsing, unit conversion, or logic tasks that can be solved without external lookup—for example, computing the total cost from a list of prices.

Use `calculate_shipping_cost` when the user asks for shipping estimates, as it applies business-specific logic and access to live rate tables. Do not attempt to estimate these using the `python` tool.

When both could be used (e.g., calculating a delivery fee), prefer `calculate_shipping_cost` for accuracy and policy compliance. Fall back to `python` only if the custom tool is unavailable or fails.
```

4. MCPについて詳しく：MCPツールの使用に関するベストプラクティスについては、より詳細な[ガイド](https://cookbook.openai.com/examples/mcp/mcp_tool_guide)がありますが、完全性のため、ここでいくつかの高レベルなガイドラインを再度説明します（これらはo3/o4-mini固有ではありませんが、依然として関連性があります）。

* ペイロードの肥大化を避けるためにツールをフィルタリングする：allowed_toolsパラメータを活用して、必要なツールのみを使用し、不要なコンテキストを節約します。MCPサーバーから返されるすべてのツールが常に必要とは限らないため、allowed_toolsフィールドを通じて必要なツールのみにフィルタリングできます。

```
    "tools": [
        {
            "type": "mcp",
            "server_label": "gitmcp",
            "server_url": "https://gitmcp.io/openai/tiktoken",
            "allowed_tools": ["search_tiktoken_documentation", "fetch_tiktoken_documentation"],
            "require_approval": "never"
        }
```

* キャッシュによるレイテンシの削減と高複雑度タスクでの推論モデルの予約：`mcp_list_tools`を返すか、`previous_response_id`を含めて、APIが不必要にツールリストを何度も再インポートする必要がないようにしてください。
* MCPを他のツールと併用する：MCPを他のホストされたツールやカスタム定義関数と組み合わせて使用できます。ツールを組み合わせる場合は、決定境界を定義し、全体的な開発者プロンプトを使用して、あるツールを別のツールより優先して使用するタイミングを明示することが有効です。MCPツールガイドの[こちら](https://cookbook.openai.com/examples/mcp/mcp_tool_guide#using-mcp-with-other-tools)に優れた例があります。

### よくある質問（FAQ）

**Q: 関数の数はどのくらいまでが適切ですか？**

**A:** o3およびo4-miniモデルでは、関数の数に厳密な上限はありませんが、トレーニングデータの分布と観察されたモデルの動作に基づいた実用的なガイダンスは存在します。2025年5月時点では、約100個未満のツールと、ツールあたり約20個未満の引数を持つ設定は分布内とみなされ、期待される信頼性の範囲内で動作するはずです。パフォーマンスは依然としてプロンプトの設計とタスクの複雑さに依存します。

技術的にはトレーニング分布内であっても、より多くのツールは曖昧さや混乱を招く可能性があります。以下が主な考慮事項です：

* 関数の説明の明確さが重要になります：複数のツールが重複する目的を持っていたり、曖昧な説明がある場合、モデルは間違ったものを呼び出したり、全く呼び出すことを躊躇したりする可能性があります。

* ツールリストのサイズはレイテンシと推論の深さに影響する可能性があります：リストが長いほど、モデルは推論フェーズでより多くの選択肢を解析する必要があります。o3/o4-miniは統合された推論パイプラインでこれを処理できますが、スキーマの明確さや呼び出し条件が明確でない場合、パフォーマンスが低下する可能性があります。

* ツールの幻覚は複雑さとともに増加する可能性があります：特にo3では、ツールセットが大きく定義が不十分な場合、幻覚的または推測的なツール呼び出しの報告があります。明示的な指示がこれを軽減するのに役立ちます（例：「ツールX、Y、Zのみを使用してください。ツール呼び出しを発明したり、将来のターンに延期したりしないでください。」）

最終的に、パフォーマンスはユースケースによって異なります。そのため、反復改善に使用できる信頼できる評価に投資することが重要です。

**Q: ツール内で深くネストしたパラメータを持つことは問題ありませんか、それともスキーマを「フラット化」すべきでしょうか？**

**A:** これについても厳密なガイダンスはありません。しかし、ネスト構造が技術的にサポートされていても、深く階層化された引数ツリーはパフォーマンスや信頼性に影響を与える可能性があります。迷った場合は、引数をフラットにする方向で判断することをお勧めします。

フラット構造はモデルにとって推論しやすいことが多いです：フラットなスキーマでは、引数フィールドがトップレベルにあり、すぐに見えます。これにより、内部解析と構造化の必要性が減り、部分的に埋められたネストオブジェクトや無効なフィールドの組み合わせなどの問題を防ぐのに役立ちます。深くネストしたオブジェクト、特に繰り返しまたは意味的に類似したフィールド名を持つものでは、モデルが引数を省略したり誤用したりする可能性が高くなります。

ネストは複雑なロジックの整理に役立ちますが、追加の注意が必要です：設定ペイロード、リッチ検索フィルター、フォーム送信など、構造化された入力を自然に含むドメインでは、ネストが関連パラメータの整理に役立ちます。ただし、無効な引数の組み合わせを防ぎ、モデルの信頼性を向上させるために、明確なフィールド説明、anyOfロジック、または厳密なスキーマなどの技術を使用する必要があります。

最適な選択方法は、独自の評価でテストし、成功を測定することです。呼び出し動作は創発的でプロンプトに敏感であるため、「万能」の解決策はありません。

**Q: この関数呼び出しのガイダンスはカスタムツール形式にも適用されますか？**

**A:** 保証されません。このドキュメントのガイダンスは、[関数呼び出しの一般ガイド](https://platform.openai.com/docs/guides/function-calling)で示されているように、標準の`tools`モデルパラメータを使用して関数スキーマを渡すことを前提としています。我々のo3/o4-miniモデルは、ツール選択と引数構築のためにこれらのスキーマをネイティブに理解し使用するようにトレーニングされています。

代わりに開発者が作成したプロンプトで自然言語を介してカスタムツール定義を提供している場合（例：開発者メッセージやユーザーメッセージでツールをインライン定義する）、このガイダンスは完全には適用されない可能性があります。そのような場合：
モデルは内部のツールスキーマの事前知識に依存していません。
few-shotの例、出力形式、ツール選択基準についてより明示的である必要があるかもしれません。
スキーマレベルの固定なしに引数構築の信頼性が低下する可能性があります。

可能な場合は構造化されたtoolsパラメータを使用してください。フリーテキストでツールを定義する必要がある場合は、それをカスタムプロトコルとして扱い、それに応じてテストしてください。